## Cuarto caso práctico de Modelos de Simulación
#### - Rivera Juan Matías

In [2]:
import datetime

##### Función transforma un datetime en un string con valor igual al tiempo(time)

In [3]:
def transformarTiempo(tiempo):
    st = datetime.datetime.strptime(str(tiempo), "%Y-%m-%d %H:%M:%S")
    return st.time()

#### Clase de cola

In [4]:
class Cola:
    def __init__(self,q,TiempoLlegada,DeltaLlegada):
        self.q=q
        self.tiempoL=TiempoLlegada
        self.deltaL = DeltaLlegada
        self.contador = 0

    def Llegada(self,PS):
        tiempoA = self.tiempoL
        if PS.estado == 0:
            PS.estado = 1
            PS.tiempoFS = self.tiempoL + PS.deltaFS[PS.contador]
            PS.contador+=1
        else:
            self.q += 1
        self.tiempoL += self.deltaL[self.contador]
        self.contador +=1
        
        return tiempoA
    
    def ReinicioContador(self):
        if self.contador > len(self.deltaL)-1:
            self.contador = 0

#### Clase de Puesto de Servicio

In [5]:
class PuestoDeServicio:
    def __init__(self,Estado,TiempoFS,DeltaFS,Limite):
        self.estado = Estado
        self.tiempoFS = TiempoFS
        self.deltaFS = DeltaFS
        self.contador = 0
        self.limite = Limite
        
    def FinDeServicio(self,ColaA,ColaB):
        tiempoA = self.tiempoFS
        
        if ColaA.q > 0:
            ColaA.q -= 1
            self.tiempoFS += self.deltaFS[self.contador]
            self.contador += 1
        elif ColaB.q > 0:
            ColaB.q -= 1
            self.tiempoFS += self.deltaFS[self.contador]
            self.contador += 1
        else:
            self.estado = 0
            self.tiempoFS = self.limite
        return tiempoA
        
    def ReinicioContador(self):
        if self.contador > len(self.deltaFS)-1:
            self.contador = 0

##### Función para imprimir Tiempo, recibe variables en el siguiente orden: Hora Actual, LLegada del Cliente, Fin de Servicio, N° Cola (q) y Estado del Puesto de Servicio(PS)'

In [6]:
def imprimirTiempo(ColaA,ColaB,PS,TiempoA):
    
    tiempoActual = transformarTiempo(TiempoA)
    llegadaA = transformarTiempo(ColaA.tiempoL)
    llegadaB = transformarTiempo(ColaB.tiempoL)
    finDeServicio = transformarTiempo(PS.tiempoFS)

    
    
    if PS.estado==0:
        finDeServicio = "        "

    print(tiempoActual," | ",llegadaA," | ",llegadaB," | ",finDeServicio," | ",ColaA.q," | ",ColaB.q," | ",PS.estado)

##### Función vector de deltas de tiempo por segundo'

In [7]:
def delta(tiemposDelta):
    listaDelta = []
    for i in tiemposDelta:
        listaDelta.append(datetime.timedelta(seconds = i))
    return listaDelta

##### Función asignación de tiempo'

In [8]:
def tiempo(horas,minutos,segundos):
    tiempo = datetime.datetime.combine(datetime.datetime.today(),datetime.time(horas,minutos,segundos,0)) 
    return tiempo

##### Declaración de los tiempos iniciales y los deltas

In [9]:
#Instancia del limite(HORA DE SERVICIO)'
limite = tiempo(12,0,0)

#Instancia de los tiempos iniciales
tiempoActual = tiempo(8,0,0)
tiempoLA = tiempo(8,3,0)
tiempoLB = tiempo(8,2,0)
tiempoFS = tiempo(8,1,0)

#Tiempos delta particulares del práctico
tiempoDLA = [80,120]
tiempoDLB = [40,30,35,50]
tiempoDFS = [30,35,40,20,30]
deltaLA = delta(tiempoDLA)
deltaLB = delta(tiempoDLB)
deltaFS = delta(tiempoDFS)

#Instancia de la Cola y del Puesto de Servicio
colaA = Cola(0,tiempoLA,deltaLA)
colaB = Cola(0,tiempoLB,deltaLB)
PS = PuestoDeServicio(1,tiempoFS,deltaFS,limite)


##### Programa principal, un while que imprime toda la matriz.

In [10]:
print(" Actual   |  LLegadaA  |  LLegadaB  |  Fin de S. |  qA |  qB |  PS |")
print("--------------------------------------------------------------------")

#Se imprime primero el estado inicial
imprimirTiempo(colaA,colaB,PS,tiempoActual)


while tiempoActual < limite:
    
        if colaA.tiempoL <= colaB.tiempoL and colaA.tiempoL <= PS.tiempoFS and colaA.tiempoL <= limite:
            tiempoActual = colaA.Llegada(PS)
            imprimirTiempo(colaA,colaB,PS,tiempoActual)
        
        
        elif colaB.tiempoL < colaA.tiempoL and colaB.tiempoL <= PS.tiempoFS and colaB.tiempoL < limite:
            tiempoActual = colaB.Llegada(PS)
            imprimirTiempo(colaA,colaB,PS,tiempoActual)
            
            
        elif PS.tiempoFS <= colaA.tiempoL and PS.tiempoFS <= colaB.tiempoL and PS.tiempoFS < limite and PS.estado == 1:
            tiempoActual = PS.FinDeServicio(colaA,colaB)
            imprimirTiempo(colaA,colaB,PS,tiempoActual)
            
        #Cierre
        else:
            if PS.estado == 1:
                print("El último cliente en el PS saldrá a las horas:",transformarTiempo(PS.tiempoFS))
            break
        
        #Reinicio de contadores
        colaA.ReinicioContador()
        colaB.ReinicioContador()
        PS.ReinicioContador()

print("Clientes sin atención- A:",colaA.q," B:",colaB.q)
    

 Actual   |  LLegadaA  |  LLegadaB  |  Fin de S. |  qA |  qB |  PS |
--------------------------------------------------------------------
08:00:00  |  08:03:00  |  08:02:00  |  08:01:00  |  0  |  0  |  1
08:01:00  |  08:03:00  |  08:02:00  |            |  0  |  0  |  0
08:02:00  |  08:03:00  |  08:02:40  |  08:02:30  |  0  |  0  |  1
08:02:30  |  08:03:00  |  08:02:40  |            |  0  |  0  |  0
08:02:40  |  08:03:00  |  08:03:10  |  08:03:15  |  0  |  0  |  1
08:03:00  |  08:04:20  |  08:03:10  |  08:03:15  |  1  |  0  |  1
08:03:10  |  08:04:20  |  08:03:45  |  08:03:15  |  1  |  1  |  1
08:03:15  |  08:04:20  |  08:03:45  |  08:03:55  |  0  |  1  |  1
08:03:45  |  08:04:20  |  08:04:35  |  08:03:55  |  0  |  2  |  1
08:03:55  |  08:04:20  |  08:04:35  |  08:04:15  |  0  |  1  |  1
08:04:15  |  08:04:20  |  08:04:35  |  08:04:45  |  0  |  0  |  1
08:04:20  |  08:06:20  |  08:04:35  |  08:04:45  |  1  |  0  |  1
08:04:35  |  08:06:20  |  08:05:15  |  08:04:45  |  1  |  1  |  1
08:0

08:47:40  |  08:49:40  |  08:47:40  |  08:47:50  |  1  |  10  |  1
08:47:40  |  08:49:40  |  08:48:30  |  08:47:50  |  1  |  11  |  1
08:47:50  |  08:49:40  |  08:48:30  |  08:48:10  |  0  |  11  |  1
08:48:10  |  08:49:40  |  08:48:30  |  08:48:40  |  0  |  10  |  1
08:48:30  |  08:49:40  |  08:49:10  |  08:48:40  |  0  |  11  |  1
08:48:40  |  08:49:40  |  08:49:10  |  08:49:10  |  0  |  10  |  1
08:49:10  |  08:49:40  |  08:49:40  |  08:49:10  |  0  |  11  |  1
08:49:10  |  08:49:40  |  08:49:40  |  08:49:45  |  0  |  10  |  1
08:49:40  |  08:51:00  |  08:49:40  |  08:49:45  |  1  |  10  |  1
08:49:40  |  08:51:00  |  08:50:15  |  08:49:45  |  1  |  11  |  1
08:49:45  |  08:51:00  |  08:50:15  |  08:50:25  |  0  |  11  |  1
08:50:15  |  08:51:00  |  08:51:05  |  08:50:25  |  0  |  12  |  1
08:50:25  |  08:51:00  |  08:51:05  |  08:50:45  |  0  |  11  |  1
08:50:45  |  08:51:00  |  08:51:05  |  08:51:15  |  0  |  10  |  1
08:51:00  |  08:53:00  |  08:51:05  |  08:51:15  |  1  |  10  

09:25:20  |  09:26:20  |  09:25:50  |  09:25:20  |  0  |  19  |  1
09:25:20  |  09:26:20  |  09:25:50  |  09:25:55  |  0  |  18  |  1
09:25:50  |  09:26:20  |  09:26:25  |  09:25:55  |  0  |  19  |  1
09:25:55  |  09:26:20  |  09:26:25  |  09:26:35  |  0  |  18  |  1
09:26:20  |  09:27:40  |  09:26:25  |  09:26:35  |  1  |  18  |  1
09:26:25  |  09:27:40  |  09:27:15  |  09:26:35  |  1  |  19  |  1
09:26:35  |  09:27:40  |  09:27:15  |  09:26:55  |  0  |  19  |  1
09:26:55  |  09:27:40  |  09:27:15  |  09:27:25  |  0  |  18  |  1
09:27:15  |  09:27:40  |  09:27:55  |  09:27:25  |  0  |  19  |  1
09:27:25  |  09:27:40  |  09:27:55  |  09:27:55  |  0  |  18  |  1
09:27:40  |  09:29:40  |  09:27:55  |  09:27:55  |  1  |  18  |  1
09:27:55  |  09:29:40  |  09:28:25  |  09:27:55  |  1  |  19  |  1
09:27:55  |  09:29:40  |  09:28:25  |  09:28:30  |  0  |  19  |  1
09:28:25  |  09:29:40  |  09:29:00  |  09:28:30  |  0  |  20  |  1
09:28:30  |  09:29:40  |  09:29:00  |  09:29:10  |  0  |  19  

10:02:45  |  10:03:00  |  10:03:25  |  10:03:05  |  0  |  26  |  1
10:03:00  |  10:04:20  |  10:03:25  |  10:03:05  |  1  |  26  |  1
10:03:05  |  10:04:20  |  10:03:25  |  10:03:35  |  0  |  26  |  1
10:03:25  |  10:04:20  |  10:04:05  |  10:03:35  |  0  |  27  |  1
10:03:35  |  10:04:20  |  10:04:05  |  10:04:05  |  0  |  26  |  1
10:04:05  |  10:04:20  |  10:04:35  |  10:04:05  |  0  |  27  |  1
10:04:05  |  10:04:20  |  10:04:35  |  10:04:40  |  0  |  26  |  1
10:04:20  |  10:06:20  |  10:04:35  |  10:04:40  |  1  |  26  |  1
10:04:35  |  10:06:20  |  10:05:10  |  10:04:40  |  1  |  27  |  1
10:04:40  |  10:06:20  |  10:05:10  |  10:05:20  |  0  |  27  |  1
10:05:10  |  10:06:20  |  10:06:00  |  10:05:20  |  0  |  28  |  1
10:05:20  |  10:06:20  |  10:06:00  |  10:05:40  |  0  |  27  |  1
10:05:40  |  10:06:20  |  10:06:00  |  10:06:10  |  0  |  26  |  1
10:06:00  |  10:06:20  |  10:06:40  |  10:06:10  |  0  |  27  |  1
10:06:10  |  10:06:20  |  10:06:40  |  10:06:40  |  0  |  26  

10:40:15  |  10:41:00  |  10:40:45  |  10:40:50  |  0  |  34  |  1
10:40:45  |  10:41:00  |  10:41:20  |  10:40:50  |  0  |  35  |  1
10:40:50  |  10:41:00  |  10:41:20  |  10:41:30  |  0  |  34  |  1
10:41:00  |  10:43:00  |  10:41:20  |  10:41:30  |  1  |  34  |  1
10:41:20  |  10:43:00  |  10:42:10  |  10:41:30  |  1  |  35  |  1
10:41:30  |  10:43:00  |  10:42:10  |  10:41:50  |  0  |  35  |  1
10:41:50  |  10:43:00  |  10:42:10  |  10:42:20  |  0  |  34  |  1
10:42:10  |  10:43:00  |  10:42:50  |  10:42:20  |  0  |  35  |  1
10:42:20  |  10:43:00  |  10:42:50  |  10:42:50  |  0  |  34  |  1
10:42:50  |  10:43:00  |  10:43:20  |  10:42:50  |  0  |  35  |  1
10:42:50  |  10:43:00  |  10:43:20  |  10:43:25  |  0  |  34  |  1
10:43:00  |  10:44:20  |  10:43:20  |  10:43:25  |  1  |  34  |  1
10:43:20  |  10:44:20  |  10:43:55  |  10:43:25  |  1  |  35  |  1
10:43:25  |  10:44:20  |  10:43:55  |  10:44:05  |  0  |  35  |  1
10:43:55  |  10:44:20  |  10:44:45  |  10:44:05  |  0  |  36  

11:46:20  |  11:47:40  |  11:46:45  |  11:46:25  |  1  |  48  |  1
11:46:25  |  11:47:40  |  11:46:45  |  11:46:55  |  0  |  48  |  1
11:46:45  |  11:47:40  |  11:47:25  |  11:46:55  |  0  |  49  |  1
11:46:55  |  11:47:40  |  11:47:25  |  11:47:25  |  0  |  48  |  1
11:47:25  |  11:47:40  |  11:47:55  |  11:47:25  |  0  |  49  |  1
11:47:25  |  11:47:40  |  11:47:55  |  11:48:00  |  0  |  48  |  1
11:47:40  |  11:49:40  |  11:47:55  |  11:48:00  |  1  |  48  |  1
11:47:55  |  11:49:40  |  11:48:30  |  11:48:00  |  1  |  49  |  1
11:48:00  |  11:49:40  |  11:48:30  |  11:48:40  |  0  |  49  |  1
11:48:30  |  11:49:40  |  11:49:20  |  11:48:40  |  0  |  50  |  1
11:48:40  |  11:49:40  |  11:49:20  |  11:49:00  |  0  |  49  |  1
11:49:00  |  11:49:40  |  11:49:20  |  11:49:30  |  0  |  48  |  1
11:49:20  |  11:49:40  |  11:50:00  |  11:49:30  |  0  |  49  |  1
11:49:30  |  11:49:40  |  11:50:00  |  11:50:00  |  0  |  48  |  1
11:49:40  |  11:51:00  |  11:50:00  |  11:50:00  |  1  |  48  